# Libs and pre-definitions

In [ ]:
a

In [1]:
import os
REPOSITORY_PATH = r'/home/luis-felipe/UncEst'
DATA_PATH = os.path.join(REPOSITORY_PATH,'data')
#CORRUPTED_DATA_PATH = os.path.join(DATA_PATH,'corrupted')

PATH_MODELS = os.path.join(REPOSITORY_PATH,'torch_models','new')
PATH_TRAINER = os.path.join(PATH_MODELS,'trainer')

### Bibliotecas padrões python e utils pytorch

In [2]:
import torch
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
from torch import nn

In [3]:
# Define o computador utilizado como cuda (gpu) se existir ou cpu caso contrário
print(torch.cuda.is_available())
dev = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

True


### Bibliotecas desenvolvidas

https://github.com/lfpc/Uncertainty_Estimation

In [4]:
import NN_models as models
import NN_utils as utils
import NN_utils.train_and_eval as TE
import torch_data

# Train classifier

## Definitions

### General

In [5]:
CREATE_DIR = True #If true, creates directories to save model (weights_path)
LIVE_PLOT = False #If True, plot* loss while training. If 'print', print loss per epoch
SAVE_CHECKPOINT = True #If True, save (and update) model weights for the best epoch (smallest validation loss)
SAVE_ALL = False #If True, saves weights and trainer at the end of training

N_EPOCHS = 250
loss_criterion = nn.CrossEntropyLoss()

### Train

In [6]:
import torchvision.transforms as transforms
MEAN = (0.4914, 0.4822, 0.4465)
STD = (0.2023, 0.1994, 0.2010)
Resize_Transform_train = transforms.Compose([
                    transforms.Resize(256),
                    transforms.RandomCrop(224),
                    transforms.RandomHorizontalFlip(),
                    transforms.ToTensor(),
                    #transforms.RandomRotation(15),
                    transforms.Normalize(MEAN, STD)])
Resize_Transform_test = transforms.Compose([
                    transforms.Resize(256),
                    transforms.CenterCrop(224),
                    transforms.ToTensor(),
                    #transforms.RandomRotation(15),
                    transforms.Normalize(MEAN, STD)])

In [7]:
#Normal mode: 91 with pytorch-cifar, 88 with pytorch

In [8]:
params = {'train_batch_size': 10, 'validation_size': 0.0, 'test_batch_size': 10}

In [ ]:
for MODEL_ARC in ['EfficientNetB0','ResNet18','ResNet50','VGG_16']:
    print(MODEL_ARC)
    for DATA in ['Cifar10','Cifar100']:
        for N_ENS in range(4):
            name = MODEL_ARC + '_' + DATA + str(N_ENS)
            data = torch_data.__dict__[DATA](data_dir = DATA_PATH, params = params)
            if 'EfficientNet' in MODEL_ARC:  
                data.change_transforms(Resize_Transform_train,Resize_Transform_test)
            num_classes = data.n_classes
            model_class = models.__dict__[MODEL_ARC]
            weights_path = os.path.join(PATH_MODELS,MODEL_ARC,DATA)

            if CREATE_DIR and not os.path.isdir(weights_path):
                os.makedirs(weights_path)

            model = model_class(num_classes = data.n_classes).to(dev)
            model.name = name
            optimizer = torch.optim.SGD(model.parameters(), lr=0.1,momentum = 0.9,weight_decay = 5e-4,nesterov = True)
            scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=N_EPOCHS)
            model_trainer = TE.Trainer(model,optimizer,loss_criterion, None,data.test_dataloader,lr_scheduler = scheduler)
            model_trainer.fit(data.train_dataloader,N_EPOCHS, live_plot = LIVE_PLOT,save_checkpoint = SAVE_CHECKPOINT,PATH = weights_path)
            print(f'model = {MODEL_ARC}, Data = {DATA}')

            model.eval()
            acc = TE.model_acc(model,data.train_dataloader)
            print('Conjunto de treinamento: acc = ', acc)
            acc = TE.model_acc(model,data.test_dataloader)
            print('Conjunto de teste: acc = ', acc)
            print(f'Conjunto de teste: MAX acc (argmax epoch) = {max(model_trainer.hist_val.acc_list)}({np.argmax(model_trainer.hist_val.acc_list)})')

            if SAVE_ALL:
                model_trainer.save_all(weights_path, PATH_TRAINER,name)

EfficientNetB0
Files already downloaded and verified
Files already downloaded and verified


Progress::   0%|          | 0/250 [00:00<?, ?it/s]

Epoch progress::   0%|          | 0/5000 [00:00<?, ?it/s]

In [ ]:
for MODEL_ARC in ['VGG16_BN','ConvNeXt_Small','EfficientNet_B0']:
    for DATA in ['Cifar10','Cifar100']:
        for N_ENS in range(4):
            name = MODEL_ARC + '_' + DATA+ '_' + str(N_ENS)
            data = torch_data.__dict__[DATA](data_dir = DATA_PATH, params = params)
            if 'EfficientNet' in MODEL_ARC:  
                data.change_transforms(Resize_Transform_train,Resize_Transform_test)
            num_classes = data.n_classes
            model_class = models.pytorch.__dict__[MODEL_ARC.lower()]
            weights_path = os.path.join(PATH_MODELS,MODEL_ARC,DATA)

            if CREATE_DIR and not os.path.isdir(weights_path):
                os.makedirs(weights_path)

            model = model_class(num_classes = data.n_classes).to(dev)
            model.name = name
            optimizer = torch.optim.SGD(model.parameters(), lr=0.1,momentum = 0.9,weight_decay = 5e-4,nesterov = True)
            scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=N_EPOCHS)
            model_trainer = TE.Trainer(model,optimizer,loss_criterion, None,data.test_dataloader,lr_scheduler = scheduler)
            model_trainer.fit(data.train_dataloader,N_EPOCHS, live_plot = LIVE_PLOT,save_checkpoint = SAVE_CHECKPOINT,PATH = weights_path)
            print(f'model = {MODEL_ARC}, Data = {DATA}')

            model.eval()
            acc = TE.model_acc(model,data.train_dataloader)
            print('Conjunto de treinamento: acc = ', acc)
            acc = TE.model_acc(model,data.test_dataloader)
            print('Conjunto de teste: acc = ', acc)
            print(f'Conjunto de teste: MAX acc (argmax epoch) = {max(model_trainer.hist_val.acc_list)}({np.argmax(model_trainer.hist_val.acc_list)})')

            if SAVE_ALL:
                model_trainer.save_all(weights_path, PATH_TRAINER,name)

### Tiny Image Net

In [ ]:
for MODEL_ARC in ['ResNet50','VGG16_BN','ResNet18','EfficientNetB0_b0','ConvNeXt_Small']:
    for DATA in ['TinyImageNet']:
        for N_ENS in range(4):
            name = MODEL_ARC + '_' + DATA + str(N_ENS)
            data = torch_data.__dict__[DATA](data_dir = DATA_PATH)
            num_classes = data.n_classes
            model_class = models.pytorch.__dict__[MODEL_ARC.lower()]
            weights_path = os.path.join(PATH_MODELS,MODEL_ARC,DATA)

            if CREATE_DIR and not os.path.isdir(weights_path):
                os.makedirs(weights_path)

            model = model_class(num_classes = data.n_classes,weights = models.pretrained_models[model_class]).to(dev)
            model.name = name
            optimizer = torch.optim.SGD(model.parameters(), lr=0.1,momentum = 0.9,weight_decay = 5e-4)
            scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=N_EPOCHS)
            model_trainer = TE.Trainer(model,optimizer,loss_criterion, None,data.test_dataloader,lr_scheduler = scheduler)
            model_trainer.fit(data.train_dataloader,N_EPOCHS, live_plot = LIVE_PLOT,save_checkpoint = SAVE_CHECKPOINT,PATH = weights_path)
            print(f'model = {MODEL_ARC}, Data = {DATA}')

            model.eval()
            acc = TE.model_acc(model,data.train_dataloader)
            print('Conjunto de treinamento: acc = ', acc)
            acc = TE.model_acc(model,data.test_dataloader)
            print('Conjunto de teste: acc = ', acc)
            print(f'Conjunto de teste: MAX acc (argmax epoch) = {max(model_trainer.hist_val.acc_list)}({np.argmax(model_trainer.hist_val.acc_list)})')

            if SAVE_ALL:
                model_trainer.save_all(weights_path, PATH_TRAINER,name)